In [1]:
import gym
import random
import tensorflow as tf
from collections import deque

from matplotlib import pyplot as plt

seed = 0
np.random.seed(seed)
random.seed(seed)

### Helper Function

In [2]:
# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    patch = plt.imshow(frames[0])
    plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=5)
    display(display_animation(anim, default_mode='loop'))

### Deep Q Learning Agent

In [11]:
lass DQNAgent:
    def __init__(self, obs_dim, n_action, seed=0,
                 discount_factor = 0.995, epsilon_decay = 0.999, epsilon_min = 0.01,
                 learning_rate = 1e-3, # Step size for Adam
                 batch_size = 64, 
                 memory_size = 2000, hidden_unit_size = 64):
        
        self.seed = seed 
        
        # Environment Information
        self.obs_dim = obs_dim
        self.n_action = n_action
        self.discount_factor = discount_factor
        
        # Epsilon Greedy Policy
        self.epsilon = 1.0
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        # Network Hyperparameters
        self.hidden_unit_size = hidden_unit_size
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.train_start = 1000

        # Experience Replay
        self.memory = deque(maxlen=memory_size)
        
        # Define Computational Graph in TF
        self.g = tf.Graph()
        with self.g.as_default():
            self.build_placeholders()
            self.build_model()
            self.build_loss()
            self.build_update_operation()
            self.init_session() # Initialize all parameters in graph
    
    def build_placeholders(self): # Build input and output place holder
        self.obs_ph = tf.placeholder(tf.float32, (None, self.obs_dim), 'obs') # Input state
        self.target_ph = tf.placeholder(tf.float32, (None, self.n_action), 'target') # TD target
        self.learning_rate_ph = tf.placeholder(tf.float32, (), 'lr')        
    
    def build_model(self): # Build networks
        hid1_size = self.hidden_unit_size
        hid2_size = self.hidden_unit_size
        
        with tf.variable_scope('q_prediction'): # Prediction Network / Two layered perceptron / Training Parameters
            out = tf.layers.dense(self.obs_ph, hid1_size, tf.tanh, # Tangent Hyperbolic Activation
                                  kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed=self.seed), name='hidden1')
            out = tf.layers.dense(out, hid2_size, tf.tanh, # Tangent Hyperbolic Activation
                                  kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed=self.seed), name='hidden2')
            self.q_predict = tf.layers.dense(out, self.n_action, # Linear Layer
                                  kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed=self.seed), name='q_predict')
                        
        with tf.variable_scope('q_target'): # Target Network / Two layered perceptron / Old Parameters
            out = tf.layers.dense(self.obs_ph, hid1_size, tf.tanh, # Tangent Hyperbolic Activation
                                  kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed=self.seed), name='hidden1')
            out = tf.layers.dense(out, hid2_size, tf.tanh, # Tangent Hyperbolic Activation
                                  kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed=self.seed), name='hidden2')
            self.q_predict_old = tf.layers.dense(out, self.n_action, # Linear Layer
                                  kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed=self.seed), name='q_predict')
        
        self.weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='q_prediction') # Get Prediction network's Parameters
        self.weights_old = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='q_target') # Get Target network's Parameters

    def build_loss(self):
        # loss는 얼마나 니가 업데이트를 할거냐(이전에 봤던 이동평균)
        # for slowly update the target network
        # prediction과 true value(target) 사이에 차이가 최소가 되는 지점(optimum)
        self.loss = 0.5*tf.reduce_mean(tf.square(self.target_ph - self.q_predict)) # Squared Error
        self.optim = tf.train.AdamOptimizer(learning_rate=self.learning_rate_ph).minimize(self.loss) # AdamOptimizer (Gradi
        
    def build_update_operation(self): 
        # parameter 업데이트
        # Define parameter update operation in TF graph
        update_ops = [] 
        for var, var_old in zip(self.weights, self.weights_old): 
            # Update Target Network's Parameter with Prediction Network
            update_ops.append(var_old.assign(var))
        self.update_ops = update_ops
        
    def init_session(self):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=config,graph=self.g) # Initialize session
        self.sess.run(tf.global_variables_initializer())
        self.sess.run(self.update_ops)

        # Summary writer
        summary_q = tf.summary.scalar('max_Q_predict', tf.reduce_max(self.q_predict))
        summary_q_old = tf.summary.scalar('max_Q_target', tf.reduce_max(self.q_predict_old))
        summary_loss = tf.summary.scalar('loss', self.loss)
        self.merge_q_step = 0
        self.merge_q = tf.summary.merge([summary_q, summary_q_old])
        self.merge_loss_step = 0
        self.merge_loss = tf.summary.merge([summary_loss])
        self.summary_writer = tf.summary.FileWriter('./tf_logs/dqn', graph=self.sess.graph)
        
    def update_policy(self):
        # epsilon을 왜 업데이트(혹은 decaying) 해주지??
        if self.epsilon > self.epsilon_min: # Update epsilon
            self.epsilon *= self.epsilon_decay
    
    def get_prediction_old(self, obs): 
        # Get Q value 
        # from target network
        q_value_old, summary = self.sess.run([self.q_predict_old,self.merge_q],feed_dict={self.obs_ph:obs}) 
        
        # Summary Q value
        self.merge_q_step += 1
        self.summary_writer.add_summary(summary,self.merge_q_step)       
        
        return q_value_old
    
    def get_prediction(self, obs): 
        # Get Q value 
        # from prediction network
        q_value, summary = self.sess.run([self.q_predict,self.merge_q],feed_dict={self.obs_ph:obs}) 
        
        # Summary Q value
        self.merge_q_step += 1
        self.summary_writer.add_summary(summary,self.merge_q_step)       
        
        return q_value
    
    def get_action(self, obs): # Epsilon Greedy policy
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.n_action)
        else:
            q_value = self.get_prediction([obs])
            return np.argmax(q_value[0])

    def add_experience(self, obs, action, reward, next_obs, done): # Add experience to memory
        self.memory.append((obs, action, reward, next_obs, done))

    def train_model(self):
        loss = np.nan
        n_entries = len(self.memory)
            
        if n_entries > self.train_start: # Start training when the number of experience is greater than batch size
            
            # Randomly sample batch
            mini_batch = random.sample(self.memory, self.batch_size)
            
            observations = np.zeros((self.batch_size, self.obs_dim))
            next_observations = np.zeros((self.batch_size, self.obs_dim))
            actions, rewards, dones = [], [], []

            for i in range(self.batch_size):
                observations[i] = mini_batch[i][0]
                actions.append(mini_batch[i][1])
                rewards.append(mini_batch[i][2])
                next_observations[i] = mini_batch[i][3]
                dones.append(mini_batch[i][4])

            target = self.get_prediction(observations)
            next_q_value = self.get_prediction_old(next_observations)

            # BELLMAN UPDATE RULE 
            for i in range(self.batch_size):
                if dones[i]:
                    target[i][actions[i]] = rewards[i]
                else:
                    target[i][actions[i]] = rewards[i] + self.discount_factor * (np.amax(next_q_value[i]))

            loss, _, summary = self.sess.run([self.loss, self.optim, self.merge_loss], 
                                 feed_dict={self.obs_ph:observations,self.target_ph:target,self.learning_rate_ph:self.learning_rate})                        
            
            # Summary loss
            self.merge_loss_step += 1
            self.summary_writer.add_summary(summary, self.merge_loss_step)       
        return loss